In [233]:
import warnings
warnings.filterwarnings(action='ignore')
import os
import pandas as pd
pd.set_option('display.max_rows', 300)
pd.set_option('display.expand_frame_repr', True)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 800)
import numpy as np
from tqdm.notebook import tqdm

In [2]:
domhotel = pd.read_parquet('/data1/DATA/resource/mkt_data/tour_ga_domhotel.parquet')

In [4]:
domhotel.shape

(4870704, 12)

In [3]:
domhotel.head()

,memberId,date,visitId,hHitnumber,hType,heActionType,hpProductId,hpHostName,hpPagepath,log_class,dDeviceCategory,hcDeviceCategory
0,THD/ryB9TaEl4LPWUzOAmg==,20220131,1643639630,56,EVENT,6,N200256924,travel.interpark.com,/checkinnow/orderConfirm/220131948408,buy,desktop,PC_Web
1,VJvXIh265d+shaPbCLZ4WQ==,20220108,1641638876,13,PAGE,0,,mtravel.interpark.com,/checkinnow/reservation/220104754788,etc,mobile,인터파크통합앱
2,VJvXIh265d+shaPbCLZ4WQ==,20220108,1641638876,17,PAGE,0,,mtravel.interpark.com,/checkinnow,etc,mobile,인터파크통합앱
3,VJvXIh265d+shaPbCLZ4WQ==,20220108,1641638876,18,EVENT,0,,mtravel.interpark.com,/checkinnow,etc,mobile,인터파크통합앱
4,VJvXIh265d+shaPbCLZ4WQ==,20220108,1641638876,19,PAGE,0,,mtravel.interpark.com,/checkinnow/category?type=category2&code=HR001...,etc,mobile,인터파크통합앱


# 구매 유저 기반 path 파악

In [45]:
buy_list = domhotel.loc[domhotel.log_class == 'buy']['memberId'].tolist()

In [49]:
domhotel.loc[domhotel['memberId'].isin(buy_list)].shape

(1337747, 12)

In [145]:
dh_buy_user = domhotel.loc[domhotel['memberId'].isin(buy_list)]

In [52]:
dh_buy_user.groupby('memberId')['date'].count().to_frame().reset_index().mean()

date    108.240715
dtype: float64

In [67]:
tmp = dh_buy_user.groupby('memberId')['date'].count().to_frame().reset_index()
tmp.loc[(tmp.date >= 50) & (tmp.date < 100)]

,memberId,date
1,++Ya9Qlx26/Pz7a2h+XKeg==,57
2,++fiFNxHEgGBZDGos/NIDw==,55
4,+/FJP7g/OCMOnzqucvfddA==,84
14,+4FtK8q5leSVPJNIc4bJ+A==,56
20,+6ZqGoCeOrJvaQ7HKnkz0w==,55
...,...,...
12347,zuo+sUj28QoZggvCWREyMw==,54
12350,zvodC6+LFqRZjaDCux8jgQ==,99
12351,zvrUCNMHmv/cUlz9Zv8N6Q==,69
12356,zxr8P3fUourShLzW3kUVUw==,62


In [57]:
def buy_user_journey(user):
    print(dh_buy_user.loc[dh_buy_user.memberId == user].shape)
    return dh_buy_user.loc[dh_buy_user.memberId == user].sort_values(['date', 'visitId', 'hHitnumber'])

작업 완료 목록(인덱스)
: 1, 2, 4, 14, 20

In [117]:
buy_user_journey('zuo+sUj28QoZggvCWREyMw==')[:100]

(54, 12)


,memberId,date,visitId,hHitnumber,hType,heActionType,hpProductId,hpHostName,hpPagepath,log_class,dDeviceCategory,hcDeviceCategory
2183386,zuo+sUj28QoZggvCWREyMw==,20220105,1641346226,10,PAGE,0,GIM019000101,mtravel.interpark.com,/checkinnow/goods/GIM019000101?startdate=20220105&enddate=20220106&roomOptions=0^2^0&npprmaxsort=min,detail,mobile,인터파크투어 앱
2183387,zuo+sUj28QoZggvCWREyMw==,20220105,1641346226,11,EVENT,2,GIM019000101,mtravel.interpark.com,/checkinnow/goods/GIM019000101?startdate=20220105&enddate=20220106&roomOptions=0^2^0&npprmaxsort=min,detail,mobile,인터파크투어 앱
2183388,zuo+sUj28QoZggvCWREyMw==,20220105,1641346226,12,PAGE,0,GIM019000101,mtravel.interpark.com,/checkinnow/goods/GIM019000101?startdate=20220116&enddate=20220117&roomOptions=0^2^0&npprmaxsort=min,detail,mobile,인터파크투어 앱
2183389,zuo+sUj28QoZggvCWREyMw==,20220105,1641346226,13,EVENT,2,GIM019000101,mtravel.interpark.com,/checkinnow/goods/GIM019000101?startdate=20220116&enddate=20220117&roomOptions=0^2^0&npprmaxsort=min,detail,mobile,인터파크투어 앱
2189707,zuo+sUj28QoZggvCWREyMw==,20220105,1641346226,14,PAGE,0,,mtravel.interpark.com,/checkinnow/room,etc,mobile,인터파크투어 앱
2189708,zuo+sUj28QoZggvCWREyMw==,20220105,1641346226,15,PAGE,0,,mtravel.interpark.com,/checkinnow/order/000017110473,etc,mobile,인터파크투어 앱
2183390,zuo+sUj28QoZggvCWREyMw==,20220105,1641346226,53,PAGE,0,GIM019000101,mtravel.interpark.com,/checkinnow/goods/GIM019000101?startdate=20220105&enddate=20220106,detail,mobile,인터파크투어 앱
2183391,zuo+sUj28QoZggvCWREyMw==,20220105,1641346226,54,PAGE,0,GIM019000101,mtravel.interpark.com,/checkinnow/goods/GIM019000101?startdate=20220105&enddate=20220106&roomOptions=0^2^0&npprmaxsort=min,detail,mobile,인터파크투어 앱
2183392,zuo+sUj28QoZggvCWREyMw==,20220105,1641346226,55,EVENT,2,GIM019000101,mtravel.interpark.com,/checkinnow/goods/GIM019000101?startdate=20220105&enddate=20220106&roomOptions=0^2^0&npprmaxsort=min,detail,mobile,인터파크투어 앱
2183393,zuo+sUj28QoZggvCWREyMw==,20220105,1641346226,56,PAGE,0,GIM019000101,mtravel.interpark.com,/checkinnow/goods/GIM019000101?startdate=20220116&enddate=20220117&roomOptions=0^2^0&npprmaxsort=min,detail,mobile,인터파크투어 앱


- 특정 path 파악

In [112]:
def find_path_startswith(path):
    print('> hType:\n',dh_buy_user.loc[dh_buy_user.hpPagepath.str.startswith(path)]['hType'].value_counts())
    print('\n> hType == PAGE\n\n', dh_buy_user.loc[(dh_buy_user.hpPagepath.str.startswith(path)&(dh_buy_user.hType == 'PAGE'))]['hcDeviceCategory'].value_counts())
    print('\n> hType == EVENT\n\n', dh_buy_user.loc[(dh_buy_user.hpPagepath.str.startswith(path)&(dh_buy_user.hType == 'EVENT'))]['hcDeviceCategory'].value_counts())
    return dh_buy_user.loc[dh_buy_user.hpPagepath.str.startswith(path)][['memberId','hType', 'hpHostName', 'hpPagepath']][:100]

In [113]:
def find_path_same(path):
    print('> hType:\n',dh_buy_user.loc[dh_buy_user.hpPagepath == path]['hType'].value_counts())
    print('\n> hType == PAGE\n\n', dh_buy_user.loc[(dh_buy_user.hpPagepath == path) & (dh_buy_user.hType == 'PAGE')]['hcDeviceCategory'].value_counts())
    print('\n> hType == EVENT\n\n', dh_buy_user.loc[(dh_buy_user.hpPagepath == path)&(dh_buy_user.hType == 'EVENT')]['hcDeviceCategory'].value_counts())
    return dh_buy_user.loc[dh_buy_user.hpPagepath == path][['memberId','hType', 'hpHostName', 'hpPagepath']][:100]

In [114]:
find_path_startswith('/checkinnow/category')

> hType:
 PAGE     59689
EVENT    12911
Name: hType, dtype: int64

> hType == PAGE

 인터파크투어 앱      31931
체크인나우 앱       13004
Mobile_Web     6351
인터파크통합앱        4599
PC_Web         3645
항공 앱            159
Name: hcDeviceCategory, dtype: int64

> hType == EVENT

 인터파크투어 앱      7696
체크인나우 앱       2506
Mobile_Web    1433
인터파크통합앱       1055
PC_Web         173
항공 앱            48
Name: hcDeviceCategory, dtype: int64


,memberId,hType,hpHostName,hpPagepath
117,X2TnytuSrX7DfIlvEqgo4A==,PAGE,mtravel.interpark.com,/checkinnow/category?type=category2&code=HR001&value=리조트&disp_q=강릉&regiondisp=42080_81502&coord=&startdate=20220108&enddate=20220109&fe_isdawncheckin=&roomOptions=0^2^0&npprmaxsort=min
118,X2TnytuSrX7DfIlvEqgo4A==,PAGE,mtravel.interpark.com,/checkinnow/category?type=category2&code=HR001&value=리조트&disp_q=강릉&regiondisp=42080_81502&coord=&startdate=20220108&enddate=20220109&fe_isdawncheckin=&roomOptions=0^2^0&npprmaxsort=min&theme=
121,X2TnytuSrX7DfIlvEqgo4A==,PAGE,mtravel.interpark.com,/checkinnow/category?type=category2&code=HR001&value=리조트&disp_q=강원전체&regiondisp=42080&coord=&startdate=20220108&enddate=20220109&fe_isdawncheckin=&roomOptions=0^2^0&npprmaxsort=min
122,X2TnytuSrX7DfIlvEqgo4A==,PAGE,mtravel.interpark.com,/checkinnow/category?type=category2&code=HR001&value=리조트&disp_q=강원전체&regiondisp=42080&coord=&startdate=20220108&enddate=20220109&fe_isdawncheckin=&roomOptions=0^2^0&npprmaxsort=min&theme=
775,zWXBe4BS690HPigBG/Qq6g==,PAGE,mtravel.interpark.com,/checkinnow/category?type=category2&code=H0001&value=호텔&disp_q=서울전체&regiondisp=42001&coord=&startdate=20220101&enddate=20220102&fe_isdawncheckin=&roomOptions=0^2^0&npprmaxsort=min
776,zWXBe4BS690HPigBG/Qq6g==,PAGE,mtravel.interpark.com,/checkinnow/category?type=category2&code=H0001&value=호텔&disp_q=서울전체&regiondisp=42001&coord=&startdate=20220101&enddate=20220102&fe_isdawncheckin=&roomOptions=0^2^0&npprmaxsort=min&theme=
777,zWXBe4BS690HPigBG/Qq6g==,EVENT,mtravel.interpark.com,/checkinnow/category?type=category2&code=H0001&value=호텔&disp_q=서울전체&regiondisp=42001&coord=&startdate=20220101&enddate=20220102&fe_isdawncheckin=&roomOptions=0^2^0&npprmaxsort=min&theme=
778,zWXBe4BS690HPigBG/Qq6g==,PAGE,mtravel.interpark.com,/checkinnow/category?type=category2&code=H0001&value=호텔&disp_q=서울전체&regiondisp=42001&coord=&startdate=20220228&enddate=20220301&fe_isdawncheckin=&roomOptions=0^2^0&npprmaxsort=min&theme=
781,zWXBe4BS690HPigBG/Qq6g==,PAGE,mtravel.interpark.com,/checkinnow/category?type=category2&code=H0001&value=호텔&disp_q=서울전체&regiondisp=42001&coord=&startdate=20220101&enddate=20220102&fe_isdawncheckin=&roomOptions=0^2^0&npprmaxsort=min
785,zWXBe4BS690HPigBG/Qq6g==,PAGE,mtravel.interpark.com,/checkinnow/category?type=category2&code=H0001&value=호텔&disp_q=서울전체&regiondisp=42001&coord=&startdate=20220101&enddate=20220102&fe_isdawncheckin=&roomOptions=0^2^0&npprmaxsort=min


In [109]:
find_path_same('/checkinnow/')

> hType:
 PAGE     1203
EVENT     907
Name: hType, dtype: int64

> hType == PAGE

 인터파크투어 앱      487
Mobile_Web    479
PC_Web        223
인터파크통합앱         9
체크인나우 앱         5
Name: hcDeviceCategory, dtype: int64

> hType == EVENT

 인터파크투어 앱      450
Mobile_Web    438
인터파크통합앱        10
체크인나우 앱         9
Name: hcDeviceCategory, dtype: int64


,memberId,hType,hpPagepath
7653,752Minz9diAOomEI7C/Rsw==,PAGE,/checkinnow/
13180,Z5rapn0hVxKRXCwFqNn0Hg==,PAGE,/checkinnow/
13181,Z5rapn0hVxKRXCwFqNn0Hg==,EVENT,/checkinnow/
17197,Z5rapn0hVxKRXCwFqNn0Hg==,PAGE,/checkinnow/
17198,Z5rapn0hVxKRXCwFqNn0Hg==,EVENT,/checkinnow/
35820,tcwx59jgNRK1m09OAmk0jg==,PAGE,/checkinnow/
35821,tcwx59jgNRK1m09OAmk0jg==,EVENT,/checkinnow/
35822,tcwx59jgNRK1m09OAmk0jg==,PAGE,/checkinnow/
35824,tcwx59jgNRK1m09OAmk0jg==,PAGE,/checkinnow/
35825,tcwx59jgNRK1m09OAmk0jg==,EVENT,/checkinnow/


In [110]:
find_path_same('/checkinnow')

> hType:
 PAGE     30715
EVENT    26196
Name: hType, dtype: int64

> hType == PAGE

 인터파크투어 앱      14598
체크인나우 앱       10208
Mobile_Web     3048
인터파크통합앱        2433
PC_Web          266
항공 앱            162
Name: hcDeviceCategory, dtype: int64

> hType == EVENT

 인터파크투어 앱      13112
체크인나우 앱        7760
Mobile_Web     2708
인터파크통합앱        2209
PC_Web          251
항공 앱            156
Name: hcDeviceCategory, dtype: int64


,memberId,hType,hpPagepath
113,X2TnytuSrX7DfIlvEqgo4A==,PAGE,/checkinnow
114,X2TnytuSrX7DfIlvEqgo4A==,EVENT,/checkinnow
115,X2TnytuSrX7DfIlvEqgo4A==,PAGE,/checkinnow
116,X2TnytuSrX7DfIlvEqgo4A==,EVENT,/checkinnow
119,X2TnytuSrX7DfIlvEqgo4A==,PAGE,/checkinnow
120,X2TnytuSrX7DfIlvEqgo4A==,EVENT,/checkinnow
772,zWXBe4BS690HPigBG/Qq6g==,PAGE,/checkinnow
773,zWXBe4BS690HPigBG/Qq6g==,EVENT,/checkinnow
774,zWXBe4BS690HPigBG/Qq6g==,EVENT,/checkinnow
779,zWXBe4BS690HPigBG/Qq6g==,PAGE,/checkinnow


# 영역 분류 진척도

- 전체 샘플 데이터: 4,870,704건
- 구매 기준 샘플 데이터: 1,337,747건
- 1차: 9개, 전체 기준 12.37%

In [146]:
main_match = [r'/checkinnow/', r'/checkinnow']
main_re = ['^/checkinnow/quickMenuSearch/']
search_re = ['^/checkinnow/search']
detail_match = [r'/checkinnow/room']
detail_re = ['^/checkinnow/goods/']
order_re = ['^/checkinnow/order/', r'/checkinnow/orderConfirm/']
comfirm_re = ['^/checkinnow/reservation/']

In [147]:
domhotel.main_area = None
domhotel.search_area = None
domhotel.detail_area = None
domhotel.order_area = None
domhotel.confirm_area = None

In [148]:
domhotel.loc[(domhotel['hpPagepath'].isin(main_match))|(domhotel['hpPagepath'].str.match('|'.join(main_re))), 'main_area'] = 1
domhotel.loc[(domhotel['hpPagepath'].str.match('|'.join(search_re))), 'search_area'] = 1
domhotel.loc[(domhotel['hpPagepath'].isin(detail_match))|(domhotel['hpPagepath'].str.match('|'.join(detail_re))), 'detail_area'] = 1
domhotel.loc[(domhotel['hpPagepath'].str.match('|'.join(order_re))), 'order_area'] = 1
domhotel.loc[(domhotel['hpPagepath'].str.match('|'.join(comfirm_re))), 'comfirm_area'] = 1

In [149]:
cols = ['main_area', 'search_area', 'detail_area', 'order_area', 'comfirm_area']
domhotel['whole_area'] = domhotel[cols].sum(axis=1)

In [152]:
domhotel.head()

,memberId,date,visitId,hHitnumber,hType,heActionType,hpProductId,hpHostName,hpPagepath,log_class,dDeviceCategory,hcDeviceCategory,main_area,search_area,detail_area,order_area,comfirm_area,whole_area
0,THD/ryB9TaEl4LPWUzOAmg==,20220131,1643639630,56,EVENT,6,N200256924,travel.interpark.com,/checkinnow/orderConfirm/220131948408,buy,desktop,PC_Web,NaN,NaN,NaN,1.0,NaN,1.0
1,VJvXIh265d+shaPbCLZ4WQ==,20220108,1641638876,13,PAGE,0,,mtravel.interpark.com,/checkinnow/reservation/220104754788,etc,mobile,인터파크통합앱,NaN,NaN,NaN,NaN,1.0,1.0
2,VJvXIh265d+shaPbCLZ4WQ==,20220108,1641638876,17,PAGE,0,,mtravel.interpark.com,/checkinnow,etc,mobile,인터파크통합앱,1.0,NaN,NaN,NaN,NaN,1.0
3,VJvXIh265d+shaPbCLZ4WQ==,20220108,1641638876,18,EVENT,0,,mtravel.interpark.com,/checkinnow,etc,mobile,인터파크통합앱,1.0,NaN,NaN,NaN,NaN,1.0
4,VJvXIh265d+shaPbCLZ4WQ==,20220108,1641638876,19,PAGE,0,,mtravel.interpark.com,/checkinnow/category?type=category2&code=HR001&value=리조트&disp_q=울진/영덕/청송&regiondisp=42030_81763|81772|81769&coord=&startdate=20220108&enddate=20220109&fe_isdawncheckin=&roomOptions=0^2^0&npprmaxsort=min,etc,mobile,인터파크통합앱,NaN,NaN,NaN,NaN,NaN,0.0


In [150]:
print('영역 구분 공란(%) : {0:.2f}%'.format(domhotel[domhotel['whole_area'] == 0].shape[0] / domhotel.shape[0] * 100) )

영역 구분 공란(%) : 12.37%


In [153]:
print('영역 중복(%) : {0:.2f}%'.format(domhotel[domhotel['whole_area'] > 1].shape[0] / domhotel.shape[0] * 100) )

영역 중복(%) : 0.00%


# 분류 안 된 로그 파악

In [154]:
tmp = domhotel[domhotel['whole_area'] == 0]
tmp = tmp.groupby('memberId')['date'].count().to_frame().reset_index()
tmp.loc[(tmp.date >= 50) & (tmp.date < 100)]

,memberId,date
3,++DPSW/ngbaCkuh5+HWzfQ==,51
66,+2TcLo+RyWxyJVmKragyzA==,79
93,+4AWktfvdpAwDnFP/mjWfA==,65
106,+4wWXgkkwnzVUOzoofFnGg==,71
121,+6/xpckbvPpBRWcZ/ee+Jw==,55
...,...,...
65800,zmcYNGXBRQgy5miY1PgCQw==,72
65812,zn78l0UJVLTE5X8ErnzC0A==,73
65884,zrrd4DT4oaJunO3n9Vdbfw==,58
65890,zsYtEWIo/piBbggofUov+Q==,51


In [193]:
tmp.shape[0]

66001

In [242]:
def not_classified_user_journey(user):
    print(domhotel.loc[(domhotel.memberId == user)].shape)
    return domhotel.loc[(domhotel.memberId == user)&(domhotel['whole_area'] == 0)][['date', 'visitId', 'hHitnumber','memberId','hType', 'hpHostName', 'hpPagepath', 'whole_area']].sort_values(['date', 'visitId', 'hHitnumber'])

In [175]:
not_classified_user_journey('zmcYNGXBRQgy5miY1PgCQw==')

(297, 18)


,memberId,hType,hpHostName,hpPagepath,whole_area
2761702,zmcYNGXBRQgy5miY1PgCQw==,PAGE,mtravel.interpark.com,/checkinnow/category?type=category2&code=H0001&value=호텔&disp_q=경주&regiondisp=42030_81753&coord=&startdate=20220129&enddate=20220131&fe_isdawncheckin=&roomOptions=0^2^0&npprmaxsort=min&theme=,0.0
2761703,zmcYNGXBRQgy5miY1PgCQw==,PAGE,mtravel.interpark.com,/checkinnow/category?type=category2&code=H0001&value=호텔&disp_q=인천전체&regiondisp=42011&coord=&startdate=20220129&enddate=20220131&fe_isdawncheckin=&roomOptions=0^2^0&npprmaxsort=min&theme=,0.0
2761704,zmcYNGXBRQgy5miY1PgCQw==,PAGE,mtravel.interpark.com,/checkinnow/category?type=category2&value=호텔&disp_q=공주/연기/계룡/세종&regiondisp=42060_81602|81867|81611|81617&coord=&startdate=20220129&enddate=20220131&fe_isdawncheckin=&npprmaxsort=min,0.0
2761705,zmcYNGXBRQgy5miY1PgCQw==,PAGE,mtravel.interpark.com,/checkinnow/category?type=category2&code=H0001&value=호텔&disp_q=공주/연기/계룡/세종&regiondisp=42060_81602|81867|81611|81617&coord=&startdate=20220129&enddate=20220131&fe_isdawncheckin=&roomOptions=0^2^0&npprmaxsort=min&theme=,0.0
2761706,zmcYNGXBRQgy5miY1PgCQw==,PAGE,mtravel.interpark.com,/checkinnow/category?type=category2&value=호텔&disp_q=제주전체&regiondisp=42090&coord=&startdate=20220129&enddate=20220131&fe_isdawncheckin=&npprmaxsort=min,0.0
2761707,zmcYNGXBRQgy5miY1PgCQw==,PAGE,mtravel.interpark.com,/checkinnow/category?type=category2&code=H0001&value=호텔&disp_q=제주전체&regiondisp=42090&coord=&startdate=20220129&enddate=20220131&fe_isdawncheckin=&roomOptions=0^2^0&npprmaxsort=min&theme=,0.0
2761709,zmcYNGXBRQgy5miY1PgCQw==,PAGE,mtravel.interpark.com,/checkinnow/category?type=category2&code=H0001&value=호텔&disp_q=제주전체&regiondisp=42090&coord=&startdate=20220129&enddate=20220131&fe_isdawncheckin=&roomOptions=0^2^0&npprmaxsort=min&theme=,0.0
2761711,zmcYNGXBRQgy5miY1PgCQw==,PAGE,mtravel.interpark.com,/checkinnow/category?type=category2&code=H0001&value=호텔&disp_q=제주전체&regiondisp=42090&coord=&startdate=20220129&enddate=20220131&fe_isdawncheckin=&roomOptions=0^2^0&npprmaxsort=min&theme=,0.0
2761696,zmcYNGXBRQgy5miY1PgCQw==,PAGE,mtravel.interpark.com,/checkinnow/category?type=category2&code=H0001&value=호텔&disp_q=인천전체&regiondisp=42011&coord=&startdate=20220101&enddate=20220102&fe_isdawncheckin=&roomOptions=0^2^0&npprmaxsort=min,0.0
2761697,zmcYNGXBRQgy5miY1PgCQw==,PAGE,mtravel.interpark.com,/checkinnow/category?type=category2&code=H0001&value=호텔&disp_q=인천전체&regiondisp=42011&coord=&startdate=20220101&enddate=20220102&fe_isdawncheckin=&roomOptions=0^2^0&npprmaxsort=min&theme=,0.0


In [181]:
def find_path_startswith(path):
    print('> hType:\n',domhotel.loc[domhotel.hpPagepath.str.startswith(path)]['hType'].value_counts())
    print('\n> hType == PAGE\n\n', domhotel.loc[(domhotel.hpPagepath.str.startswith(path)&(domhotel.hType == 'PAGE'))]['hcDeviceCategory'].value_counts())
    print('\n> hType == EVENT\n\n', domhotel.loc[(domhotel.hpPagepath.str.startswith(path)&(domhotel.hType == 'EVENT'))]['hcDeviceCategory'].value_counts())
    return domhotel.loc[domhotel.hpPagepath.str.startswith(path)][['memberId','hType', 'hpHostName', 'hpPagepath']][:100]

In [182]:
def find_path_same(path):
    print('> hType:\n',domhotel.loc[domhotel.hpPagepath == path]['hType'].value_counts())
    print('\n> hType == PAGE\n\n', domhotel.loc[(domhotel.hpPagepath == path) & (domhotel.hType == 'PAGE')]['hcDeviceCategory'].value_counts())
    print('\n> hType == EVENT\n\n', domhotel.loc[(domhotel.hpPagepath == path)&(domhotel.hType == 'EVENT')]['hcDeviceCategory'].value_counts())
    return domhotel.loc[domhotel.hpPagepath == path][['memberId','hType', 'hpHostName', 'hpPagepath']][:100]

In [192]:
find_path_startswith('/checkinnow/category')

> hType:
 PAGE     279781
EVENT     60913
Name: hType, dtype: int64

> hType == PAGE

 인터파크투어 앱      144921
체크인나우 앱        65953
Mobile_Web     30949
인터파크통합앱        26502
PC_Web          9967
항공 앱            1489
Name: hcDeviceCategory, dtype: int64

> hType == EVENT

 인터파크투어 앱      33683
체크인나우 앱       13216
Mobile_Web     7259
인터파크통합앱        5855
PC_Web          509
항공 앱            391
Name: hcDeviceCategory, dtype: int64


,memberId,hType,hpHostName,hpPagepath
4,VJvXIh265d+shaPbCLZ4WQ==,PAGE,mtravel.interpark.com,/checkinnow/category?type=category2&code=HR001&value=리조트&disp_q=울진/영덕/청송&regiondisp=42030_81763|81772|81769&coord=&startdate=20220108&enddate=20220109&fe_isdawncheckin=&roomOptions=0^2^0&npprmaxsort=min
5,VJvXIh265d+shaPbCLZ4WQ==,PAGE,mtravel.interpark.com,/checkinnow/category?type=category2&code=HR001&value=리조트&disp_q=울진/영덕/청송&regiondisp=42030_81763|81772|81769&coord=&startdate=20220108&enddate=20220109&fe_isdawncheckin=&roomOptions=0^2^0&npprmaxsort=min&theme=
6,VJvXIh265d+shaPbCLZ4WQ==,EVENT,mtravel.interpark.com,/checkinnow/category?type=category2&code=HR001&value=리조트&disp_q=울진/영덕/청송&regiondisp=42030_81763|81772|81769&coord=&startdate=20220108&enddate=20220109&fe_isdawncheckin=&roomOptions=0^2^0&npprmaxsort=min&theme=
7,VJvXIh265d+shaPbCLZ4WQ==,PAGE,mtravel.interpark.com,/checkinnow/category?type=category2&code=HR001&value=리조트&disp_q=울진/영덕/청송&regiondisp=42030_81763|81772|81769&coord=&startdate=20220111&enddate=20220112&fe_isdawncheckin=&roomOptions=0^2^0&npprmaxsort=min&theme=
9,VJvXIh265d+shaPbCLZ4WQ==,PAGE,mtravel.interpark.com,/checkinnow/category?type=category2&code=HR001&value=리조트&disp_q=울진/영덕/청송&regiondisp=42030_81763|81772|81769&coord=&startdate=20220111&enddate=20220112&fe_isdawncheckin=&roomOptions=0^2^0&npprmaxsort=min&theme=
30,1rUCY7Gni0Zd7F+NdHBh2g==,PAGE,mtravel.interpark.com,/checkinnow/category?type=category2&code=HR001&value=리조트&disp_q=울산전체&regiondisp=42091&coord=&startdate=20220108&enddate=20220109&fe_isdawncheckin=&roomOptions=0^2^0&npprmaxsort=min
31,1rUCY7Gni0Zd7F+NdHBh2g==,PAGE,mtravel.interpark.com,/checkinnow/category?type=category2&code=HR001&value=리조트&disp_q=울산전체&regiondisp=42091&coord=&startdate=20220108&enddate=20220109&fe_isdawncheckin=&roomOptions=0^2^0&npprmaxsort=min&theme=
32,1rUCY7Gni0Zd7F+NdHBh2g==,EVENT,mtravel.interpark.com,/checkinnow/category?type=category2&code=HR001&value=리조트&disp_q=울산전체&regiondisp=42091&coord=&startdate=20220108&enddate=20220109&fe_isdawncheckin=&roomOptions=0^2^0&npprmaxsort=min&theme=
33,1rUCY7Gni0Zd7F+NdHBh2g==,PAGE,mtravel.interpark.com,/checkinnow/category?type=category2&code=HR001&value=리조트&disp_q=울산전체&regiondisp=42091&coord=&startdate=20220130&enddate=20220131&fe_isdawncheckin=&roomOptions=0^2^0&npprmaxsort=min&theme=
34,1rUCY7Gni0Zd7F+NdHBh2g==,PAGE,mtravel.interpark.com,/checkinnow/category?type=category2&value=리조트&disp_q=경북전체&regiondisp=42030&coord=&startdate=20220130&enddate=20220131&fe_isdawncheckin=&npprmaxsort=min


In [188]:
decide_yet = ['/checkinnow/category', '/checkinnow/goods?', '/checkinnow?itemId=', '/checkinnow/map', '/checkinnow/block?itemId=']

In [189]:
domhotel.loc[(domhotel.hpPagepath.str.match('|'.join(decide_yet))) & (domhotel['whole_area'] == 0)].sort_values(['date', 'visitId', 'hHitnumber'])[['memberId','hType', 'hpHostName', 'hpPagepath', 'whole_area']].shape[0]

451655

In [180]:
domhotel.shape[0]

4870704

In [191]:
451655 / 4870704 *100

9.272889504268788

# 영역 분류 진척도 2차

- 전체 샘플 데이터: 4,870,704건
- 1차 미분류 샘플 데이터: 66,001건
- 2차: 추가 3개 총 12개, 전체 기준 2.01%

In [194]:
main_match = [r'/checkinnow/', r'/checkinnow']
main_re = ['^/checkinnow/quickMenuSearch/']
search_re = ['^/checkinnow/search', '^/checkinnow/category']
detail_match = [r'/checkinnow/room']
detail_re = ['^/checkinnow/goods/', '^/checkinnow/goods?', '^/checkinnow/room']
order_re = ['^/checkinnow/order/', r'/checkinnow/orderConfirm/']
comfirm_re = ['^/checkinnow/reservation/']

In [195]:
domhotel.loc[(domhotel['hpPagepath'].isin(main_match))|(domhotel['hpPagepath'].str.match('|'.join(main_re))), 'main_area'] = 1
domhotel.loc[(domhotel['hpPagepath'].str.match('|'.join(search_re))), 'search_area'] = 1
domhotel.loc[(domhotel['hpPagepath'].isin(detail_match))|(domhotel['hpPagepath'].str.match('|'.join(detail_re))), 'detail_area'] = 1
domhotel.loc[(domhotel['hpPagepath'].str.match('|'.join(order_re))), 'order_area'] = 1
domhotel.loc[(domhotel['hpPagepath'].str.match('|'.join(comfirm_re))), 'comfirm_area'] = 1

In [196]:
cols = ['main_area', 'search_area', 'detail_area', 'order_area', 'comfirm_area']
domhotel['whole_area'] = domhotel[cols].sum(axis=1)

In [197]:
domhotel.head()

,memberId,date,visitId,hHitnumber,hType,heActionType,hpProductId,hpHostName,hpPagepath,log_class,dDeviceCategory,hcDeviceCategory,main_area,search_area,detail_area,order_area,comfirm_area,whole_area
0,THD/ryB9TaEl4LPWUzOAmg==,20220131,1643639630,56,EVENT,6,N200256924,travel.interpark.com,/checkinnow/orderConfirm/220131948408,buy,desktop,PC_Web,NaN,NaN,NaN,1.0,NaN,1.0
1,VJvXIh265d+shaPbCLZ4WQ==,20220108,1641638876,13,PAGE,0,,mtravel.interpark.com,/checkinnow/reservation/220104754788,etc,mobile,인터파크통합앱,NaN,NaN,NaN,NaN,1.0,1.0
2,VJvXIh265d+shaPbCLZ4WQ==,20220108,1641638876,17,PAGE,0,,mtravel.interpark.com,/checkinnow,etc,mobile,인터파크통합앱,1.0,NaN,NaN,NaN,NaN,1.0
3,VJvXIh265d+shaPbCLZ4WQ==,20220108,1641638876,18,EVENT,0,,mtravel.interpark.com,/checkinnow,etc,mobile,인터파크통합앱,1.0,NaN,NaN,NaN,NaN,1.0
4,VJvXIh265d+shaPbCLZ4WQ==,20220108,1641638876,19,PAGE,0,,mtravel.interpark.com,/checkinnow/category?type=category2&code=HR001&value=리조트&disp_q=울진/영덕/청송&regiondisp=42030_81763|81772|81769&coord=&startdate=20220108&enddate=20220109&fe_isdawncheckin=&roomOptions=0^2^0&npprmaxsort=min,etc,mobile,인터파크통합앱,NaN,1.0,NaN,NaN,NaN,1.0


In [198]:
print('영역 구분 공란(%) : {0:.2f}%'.format(domhotel[domhotel['whole_area'] == 0].shape[0] / domhotel.shape[0] * 100) )

영역 구분 공란(%) : 2.01%


In [199]:
print('영역 중복(%) : {0:.2f}%'.format(domhotel[domhotel['whole_area'] > 1].shape[0] / domhotel.shape[0] * 100) )

영역 중복(%) : 0.00%


# 분류 안 된 로그 파악 2차

In [243]:
def user_journey(user):
    print(domhotel.loc[(domhotel.memberId == user)].shape)
    return domhotel.loc[(domhotel.memberId == user)][['date', 'visitId', 'hHitnumber','memberId','hType', 'hpHostName', 'hpPagepath', 'whole_area']].sort_values(['date', 'visitId', 'hHitnumber'])

In [200]:
tmp = domhotel[domhotel['whole_area'] == 0]
tmp = tmp.groupby('memberId')['date'].count().to_frame().reset_index()
tmp.loc[(tmp.date >= 50) & (tmp.date < 100)]

,memberId,date
1483,1JKBXmQtGzyfAauijwgr7A==,57
2060,2YDXJGcq8fdo5oDD2d630w==,72
3155,4yESc9k58yT4wTed1cJiHg==,94
4340,7Vao3UNLEEZtsixka9JObw==,50
4488,7tCDz157bMD6ofaKZat6lA==,53
6222,Bf/ru2yFOP7OOHm0jLoJRw==,94
7516,EUUjPahCgnsBBEidZkZZFg==,54
7525,EVgu6SQDA44RWuImZnNFkw==,53
7936,FSzdFMxu4iQg2dKh0mKXHw==,53
8173,G+oVe86jNpCgPXmyGqveQA==,51


In [201]:
tmp.shape

(28936, 2)

In [209]:
domhotel['hHitnumber'] = domhotel['hHitnumber'].astype('int')

In [253]:
user_journey('pXWvgCJ9e/SJdxdiprFckg==')[:300]

(144, 18)


,date,visitId,hHitnumber,memberId,hType,hpHostName,hpPagepath,whole_area
108934,20220108,1641618308,19,pXWvgCJ9e/SJdxdiprFckg==,PAGE,mtravel.interpark.com,/checkinnow/goods/GI0002001543,1.0
108935,20220108,1641618308,20,pXWvgCJ9e/SJdxdiprFckg==,PAGE,mtravel.interpark.com,/checkinnow/goods/GI0002001543?roomOptions=0^2^0&startdate=20220302&enddate=20220303&npprmaxsort=min,1.0
108936,20220108,1641618308,21,pXWvgCJ9e/SJdxdiprFckg==,EVENT,mtravel.interpark.com,/checkinnow/goods/GI0002001543?roomOptions=0^2^0&startdate=20220302&enddate=20220303&npprmaxsort=min,1.0
108375,20220122,1642843910,1,pXWvgCJ9e/SJdxdiprFckg==,PAGE,mtravel.interpark.com,/checkinnow,1.0
108376,20220122,1642843910,2,pXWvgCJ9e/SJdxdiprFckg==,EVENT,mtravel.interpark.com,/checkinnow,1.0
108377,20220122,1642843910,3,pXWvgCJ9e/SJdxdiprFckg==,PAGE,mtravel.interpark.com,/checkinnow/search,1.0
108378,20220122,1642843910,4,pXWvgCJ9e/SJdxdiprFckg==,EVENT,mtravel.interpark.com,/checkinnow/search,1.0
108379,20220122,1642843910,5,pXWvgCJ9e/SJdxdiprFckg==,EVENT,mtravel.interpark.com,/checkinnow/search,1.0
108380,20220122,1642843910,6,pXWvgCJ9e/SJdxdiprFckg==,EVENT,mtravel.interpark.com,/checkinnow/search,1.0
108381,20220122,1642843910,7,pXWvgCJ9e/SJdxdiprFckg==,EVENT,mtravel.interpark.com,/checkinnow/search,1.0


In [240]:
find_path_startswith('/checkinnow?lon=')

> hType:
 PAGE     2616
EVENT    1097
Name: hType, dtype: int64

> hType == PAGE

 인터파크투어 앱      1297
체크인나우 앱        858
인터파크통합앱        293
Mobile_Web     166
항공 앱             2
Name: hcDeviceCategory, dtype: int64

> hType == EVENT

 인터파크투어 앱      524
체크인나우 앱       370
인터파크통합앱       125
Mobile_Web     78
Name: hcDeviceCategory, dtype: int64


,memberId,hType,hpHostName,hpPagepath
7178,C2RuEXfoTqO0zW2CQtXxWQ==,PAGE,mtravel.interpark.com,/checkinnow?lon=&lat=&flag=H&mode=list
7179,C2RuEXfoTqO0zW2CQtXxWQ==,PAGE,mtravel.interpark.com,/checkinnow?lon=&lat=&flag=H&mode=list
17149,KqD2cnAERON2j/Qp/HnVmA==,PAGE,mtravel.interpark.com,/checkinnow?lon=126.13118&lat=36.68511&flag=H&mode=detail
17150,KqD2cnAERON2j/Qp/HnVmA==,PAGE,mtravel.interpark.com,/checkinnow?lon=126.13118&lat=36.68511&flag=H&mode=detail
31074,/hZddM3Xq88n8NkaUC4ysw==,PAGE,mtravel.interpark.com,/checkinnow?lon=129.12001&lat=37.492325&flag=H&mode=detail
31075,/hZddM3Xq88n8NkaUC4ysw==,EVENT,mtravel.interpark.com,/checkinnow?lon=129.12001&lat=37.492325&flag=H&mode=detail
37783,v0eXphm/Yj9esqmgG5fE/w==,PAGE,mtravel.interpark.com,/checkinnow?lon=128.51749&lat=37.21817&flag=H&mode=detail
37784,v0eXphm/Yj9esqmgG5fE/w==,PAGE,mtravel.interpark.com,/checkinnow?lon=128.51749&lat=37.21817&flag=H&mode=detail
37795,v0eXphm/Yj9esqmgG5fE/w==,PAGE,mtravel.interpark.com,/checkinnow?lon=128.51749&lat=37.21817&flag=H&mode=detail
40944,gpC3+TZI3vNH49qErAFPyw==,PAGE,mtravel.interpark.com,/checkinnow?lon=128.38147&lat=37.601536&flag=H&mode=detail


# 영역 분류 진척도 3차

- 전체 샘플 데이터: 4,870,704건
- 2차 미분류 샘플 데이터: 28936건
- 3차: 추가 3개 총 15개, 전체 기준 0.96%

In [262]:
main_match = [r'/checkinnow/', r'/checkinnow']
main_re = ['^/checkinnow/quickMenuSearch/']
search_re = ['^/checkinnow/search', '^/checkinnow/category', '^/checkinnow/oldmap', '^/checkinnow/map', '^/checkinnow?lon=']
detail_match = [r'/checkinnow/room']
detail_re = ['^/checkinnow/goods/', '^/checkinnow/goods?', '^/checkinnow/room']
order_re = ['^/checkinnow/order/', r'/checkinnow/orderConfirm/']
comfirm_re = ['^/checkinnow/reservation']

In [263]:
domhotel.loc[(domhotel['hpPagepath'].isin(main_match))|(domhotel['hpPagepath'].str.match('|'.join(main_re))), 'main_area'] = 1
domhotel.loc[(domhotel['hpPagepath'].str.match('|'.join(search_re))), 'search_area'] = 1
domhotel.loc[(domhotel['hpPagepath'].isin(detail_match))|(domhotel['hpPagepath'].str.match('|'.join(detail_re))), 'detail_area'] = 1
domhotel.loc[(domhotel['hpPagepath'].str.match('|'.join(order_re))), 'order_area'] = 1
domhotel.loc[(domhotel['hpPagepath'].str.match('|'.join(comfirm_re))), 'comfirm_area'] = 1

In [264]:
cols = ['main_area', 'search_area', 'detail_area', 'order_area', 'comfirm_area']
domhotel['whole_area'] = domhotel[cols].sum(axis=1)

In [265]:
domhotel.head()

,memberId,date,visitId,hHitnumber,hType,heActionType,hpProductId,hpHostName,hpPagepath,log_class,dDeviceCategory,hcDeviceCategory,main_area,search_area,detail_area,order_area,comfirm_area,whole_area
0,THD/ryB9TaEl4LPWUzOAmg==,20220131,1643639630,56,EVENT,6,N200256924,travel.interpark.com,/checkinnow/orderConfirm/220131948408,buy,desktop,PC_Web,NaN,NaN,NaN,1.0,NaN,1.0
1,VJvXIh265d+shaPbCLZ4WQ==,20220108,1641638876,13,PAGE,0,,mtravel.interpark.com,/checkinnow/reservation/220104754788,etc,mobile,인터파크통합앱,NaN,NaN,NaN,NaN,1.0,1.0
2,VJvXIh265d+shaPbCLZ4WQ==,20220108,1641638876,17,PAGE,0,,mtravel.interpark.com,/checkinnow,etc,mobile,인터파크통합앱,1.0,NaN,NaN,NaN,NaN,1.0
3,VJvXIh265d+shaPbCLZ4WQ==,20220108,1641638876,18,EVENT,0,,mtravel.interpark.com,/checkinnow,etc,mobile,인터파크통합앱,1.0,NaN,NaN,NaN,NaN,1.0
4,VJvXIh265d+shaPbCLZ4WQ==,20220108,1641638876,19,PAGE,0,,mtravel.interpark.com,/checkinnow/category?type=category2&code=HR001&value=리조트&disp_q=울진/영덕/청송&regiondisp=42030_81763|81772|81769&coord=&startdate=20220108&enddate=20220109&fe_isdawncheckin=&roomOptions=0^2^0&npprmaxsort=min,etc,mobile,인터파크통합앱,NaN,1.0,NaN,NaN,NaN,1.0


In [266]:
print('영역 구분 공란(%) : {0:.2f}%'.format(domhotel[domhotel['whole_area'] == 0].shape[0] / domhotel.shape[0] * 100) )

영역 구분 공란(%) : 0.96%
